In [1]:
import numpy as np
import pandas as pd
from numba import njit, prange
from interpolation import interp

In [2]:
γ = 0.5772156649

In [8]:
f = open("/Users/zejinshi/Dropbox/UArizona/ECON 696/data_for_bellman_computations.csv")
lines = f.readlines()

In [27]:
n_naics_recode = 7
n_region = 10
n_lag_inv = 2
n_lag2_inv = 2
n_ordered_violator = 3
n_DAV = 20

simulated_dist = np.empty((n_naics_recode, n_region, n_lag_inv, n_lag2_inv, n_ordered_violator, n_DAV, 80, 7))

for line in lines[1:]:
    line = line.split(',')
    for i in range(80):
        simulated_dist[int(line[0])-1, int(line[1])-1, int(line[2]), int(line[3]), int(line[4]), int(line[5])-1][i] = [float(e) for e in line[7+7*i:14+7*i]]

In [225]:
def BGL(θ):

    θ_I, θ_F, θ_V, θ_H, θ_X = θ
    
    # depreciation rate
    δ = 0.1
    
    # discount factor
    β = 0.95 ** (1/4.)
    
    # focus on the fist sector
    sector = 0
    
    # initialize V(Ω') and V(Ω)
    # set the initial values to be 0

    # region (10) * lag_investment (2) * lag2_investent(2) * ordered_violator (3) * disc_violations (20)
    n_region = 10
    n_Linv = 2
    n_L2inv = 2
    n_ordered_violator = 3
    n_DAV = 20
    
    DAV_range = np.arange(0., 10.0, 0.5)
    
    def initialize():
        # V(Ω')
        VΩ_prime = np.zeros((n_region, n_Linv, n_L2inv, n_ordered_violator, n_DAV))
        # V(Ω) 
        VΩ = np.zeros((n_region, n_Linv, n_L2inv, n_ordered_violator, n_DAV))
        
        return VΩ_prime, VΩ
    
    @njit(parallel=True)
    def VΩ_prime_operator(VΩ_prime, VΩ):
        
        for region in prange(n_region):
            for Linv in prange(n_Linv):
                for L2inv in prange(n_L2inv):
                    for ordered_violator in prange(n_ordered_violator):
                        for i_DAV in prange(n_DAV):
                            DAV = (i_DAV - 1) * 0.5
                            
                            # for compliance
                            if ordered_violator == 0:
                                VΩ_prime[region, Linv, L2inv, ordered_violator, i_DAV] = β * VΩ[region, 0, Linv, ordered_violator, i_DAV] + γ
                            
                            else:
                                
                                DAV_next = (1 - δ) * DAV

                                VΩ_prime[region, Linv, L2inv, ordered_violator, i_DAV] = np.log(np.exp(β * interp(DAV_range, VΩ[region, 0, Linv, ordered_violator, :], DAV_next))
                                                                                               + np.exp(θ_X + β * interp(DAV_range, VΩ[region, 1, Linv, ordered_violator, :], DAV_next))) + γ



#     @njit(parallel=True)
    def VΩ_operator(VΩ_prime, VΩ):
        
        for region in prange(n_region):
            for Linv in prange(n_Linv):
                for L2inv in prange(n_L2inv):
                    for ordered_violator in prange(n_ordered_violator):
                        for i_DAV in prange(n_DAV):
                            DAV = (i_DAV - 1) * 0.5
                            
                            # find out the transitions
                            # for compliance
                            if (ordered_violator == 0):
                                transition = simulated_dist[sector, region, 0, 0, 0, 0]
                            else:
                                transition = simulated_dist[sector, region, Linv, L2inv, ordered_violator, i_DAV]
                            
                            VΩ[region, Linv, L2inv, ordered_violator, i_DAV] = 0.
                            
                            for i in prange(80):
                                ins, vio, fine, prob, prob0, prob1, prob2 = transition[i]
                                util = θ_I * ins + fine * θ_F + vio * θ_V + (ordered_violator == 2) * θ_H
                                
                                # if transit to compliance plant
                                util += prob0 * VΩ_prime[region, Linv, L2inv, 0, 0]
                                
                                # if transit to regular violator or high priority violator
                                DAV_updated = DAV + vio
                                
                                util += prob1 * interp(DAV_range, VΩ_prime[region, Linv, L2inv, 1, :], DAV_updated)
                                util += prob2 * interp(DAV_range, VΩ_prime[region, Linv, L2inv, 2, :], DAV_updated)
                                
                                VΩ[region, Linv, L2inv, ordered_violator, i_DAV] += prob * util
                        

    return initialize, VΩ_prime_operator, VΩ_operator

In [226]:
initialize, VΩ_prime_operator, VΩ_operator = BGL((-2., -0.5, -0.5, -0.1, -0.1))

In [227]:
VΩ_prime, VΩ = initialize()

In [322]:
VΩ_operator(VΩ_prime, VΩ)

In [323]:
VΩ[1, 0, 0, :, 8]

array([ 9.29666323, 14.94964574,  7.92605807])

In [324]:
VΩ_prime_operator(VΩ_prime, VΩ)

In [267]:
VΩ_prime

array([[[[[  2.85253542,   2.85184119,   2.85113364, ...,
             2.8199563 ,   2.81505601,   2.80994217],
          [  6.65998644,   6.30975274,   5.95954424, ...,
            -8.7807746 , -11.16637882, -13.74908309],
          [  4.0724823 ,   3.71496339,   3.39208084, ...,
           -11.79469975, -14.20556371, -16.81413633]],

         [[  2.85253542,   2.85184119,   2.85113364, ...,
             2.8199563 ,   2.81505601,   2.80994217],
          [  6.65998644,   6.30975274,   5.95954424, ...,
            -8.7807746 , -11.16637882, -13.74908309],
          [  4.0724823 ,   3.71496339,   3.39208084, ...,
           -11.79469975, -14.20556371, -16.81413633]]],


        [[[  2.85253542,   2.85184119,   2.85113364, ...,
             2.8199563 ,   2.81505601,   2.80994217],
          [  6.18313243,   5.80689043,   5.43102555, ...,
            -8.83045092, -11.17894836, -13.72695673],
          [  3.98555191,   3.61729677,   3.27426129, ...,
           -13.11493857, -15.54340156, -

In [205]:
for i in range(100):
    
    VΩ_prime_old = np.copy(VΩ_prime)

    VΩ_operator(VΩ_prime, VΩ)
    VΩ_prime_operator(VΩ_prime, VΩ)
    
    dist = np.linalg.norm(VΩ_prime_old - VΩ_prime)
    print(dist)

137.3599063874884
108.87978827603861
83.88634584050205
63.51348816233817
47.80152815999957
36.55851131035317
29.610658786094884
26.6884352025995
27.07021950443249
29.607660931026082
33.25143932715162
37.324483892567876
41.45476100256711
45.45024850434031
49.215818681343485
52.707919321647495
55.91081181780101
58.82397629427704
61.45525422767764
63.81700194300756
65.92387413466577
67.79151961581995
69.43580395584934
70.87234516535514
72.11623993499536
73.1819081780632
74.08301212373183
74.8324228393898
75.44221702539511
75.92369304079898
76.28739895369262
76.54316786241836
76.70015733007314
76.76689082974579
76.75129980484763
76.66076542808885
76.50215947068955
76.28188391969057
76.00590913818009
75.67981047239688
75.30880328557576
74.89777645089266
74.45132437173501
73.97377762148199
73.46923231020735
72.94157829450714
72.39452635045532
71.83163442955178
71.25633311399622
70.67195038103782
70.08173577661245
69.48888408579867
68.89655857170028
68.30791383467599
67.72611832013061
67.1543

In [137]:
dist = 0.
while dist > 1e-7:
    
    VΩ_prime_old = np.copy(VΩ_prime)

    VΩ_prime_operator(VΩ_prime, VΩ)
    VΩ_operator(VΩ_prime, VΩ)
    
    dist = np.linalg.norm(VΩ_prime_old - VΩ_prime)
    print(dist)

In [113]:
VΩ_prime

array([[[[[ 41.84297502,  41.82135946,  41.79951029, ...,
            41.4518976 ,  41.43720349,  41.42330871],
          [379.01754814, 364.66464183, 350.31173553, ...,
           157.51463347, 150.90100929, 144.77734044],
          [115.111595  , 127.14563733, 139.17967967, ...,
           156.23737364, 150.82237444, 145.27235948]],

         [[ 41.84297502,  41.82135946,  41.79951029, ...,
            41.4518976 ,  41.43720349,  41.42330871],
          [379.01754814, 364.66464183, 350.31173553, ...,
           157.51463347, 150.90100929, 144.77734044],
          [115.111595  , 127.14563733, 139.17967967, ...,
           156.23737364, 150.82237444, 145.27235948]]],


        [[[ 40.93915765,  40.91361993,  40.88814489, ...,
            40.58082073,  40.56897105,  40.5576588 ],
          [352.0455364 , 335.68422293, 319.32290946, ...,
           152.65116953, 147.74962789, 142.69109526],
          [130.63980445, 141.9478855 , 153.25596654, ...,
           155.80578436, 150.24174561, 1